In [40]:
import numpy as np

# --- Daten ---
rng = np.random.default_rng()
X = rng.uniform(-1, 1, (200, 3))           # 200 Punkte, features: a,b,c, Werte zwischen -1 und +1
a, b, c = X.T

d_star = 2*a - b + 0.5*c                   # linear
e_star = b**3 - 2*c + 3*a                  # nichtlinear

d = (d_star > 0).astype(int)               # binär aus linearer Regression
e = (e_star > 0.2).astype(int)             # binär aus nichtlinearer Regression
T = np.stack([d, e], axis=1)               # (N, 2)

# Bias anhängen
Xb = np.hstack([X, np.ones((len(X),1))])   # (N, 4)

# --- Perzeptron (klassische Lernregel) ---
W = rng.normal(scale=0.01, size=(Xb.shape[1], 2))  # (4,2)
lr = 1.0
epochs = 30

learn = True
if learn:
    for _ in range(epochs):
        for i in rng.permutation(len(Xb)):
            x = Xb[i]                                # (4,)
            t = T[i]                                 # (2,)
            y = (x @ W > 0).astype(int)              # Schritt-Aktivierung
            W += lr * np.outer(x, (t - y))           # Perzeptron-Update

# --- Auswertung ---
Y = (Xb @ W > 0).astype(int)
acc_d = (Y[:,0] == d).mean()
acc_e = (Y[:,1] == e).mean()
print(f"Genauigkeit: d={acc_d:.3f} (linear), e={acc_e:.3f} (nichtlinear)")

# Ein paar Beispiele
print("\n a     b     c    | d*     d | e*     e | d_pred e_pred")
for i in range(8):
    print(f"{a[i]:+5.2f} {b[i]:+5.2f} {c[i]:+5.2f} | "
          f"{d_star[i]:+5.2f}  {d[i]} | {e_star[i]:+5.2f}  {e[i]} |   {Y[i,0]}      {Y[i,1]}")


Genauigkeit: d=1.000 (linear), e=0.985 (nichtlinear)

 a     b     c    | d*     d | e*     e | d_pred e_pred
+0.22 +0.04 +0.71 | +0.77  1 | -0.75  0 |   1      0
+0.26 -0.77 +0.04 | +1.32  1 | +0.26  1 |   1      1
+0.68 +0.06 +0.95 | +1.78  1 | +0.16  0 |   1      0
-0.24 -0.55 -0.18 | -0.02  0 | -0.53  0 |   0      0
+0.43 +0.85 +0.80 | +0.41  1 | +0.29  1 |   1      0
-0.95 +1.00 -0.54 | -3.18  0 | -0.78  0 |   0      0
+0.82 -0.37 -0.46 | +1.78  1 | +3.31  1 |   1      1
-0.43 +0.43 +0.77 | -0.91  0 | -2.75  0 |   0      0


In [3]:
X.shape

(200, 3)

In [5]:
Xb.shape

(200, 4)

In [6]:
T.shape

(200, 2)

In [7]:
T[:15]

array([[0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 1],
       [1, 0]])

In [9]:
W.shape

(4, 2)

In [10]:
W

array([[-0.0065284 ,  0.00561225],
       [-0.00588178,  0.00372526],
       [-0.00731129, -0.00824381],
       [-0.00278349, -0.02776218]])

In [11]:
Y

array([[1, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [1,

In [12]:
Y[0:5,0]

array([1, 1, 1, 0, 0])

In [14]:
d[:5]

array([1, 0, 0, 0, 1])

In [15]:
Y[:,0] == d

array([ True, False, False,  True, False, False, False,  True,  True,
       False,  True,  True,  True, False, False,  True, False, False,
       False,  True,  True, False, False, False, False,  True, False,
       False, False, False, False, False,  True,  True,  True, False,
        True, False, False, False, False,  True,  True, False,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False, False,  True,  True, False, False,  True, False,  True,
       False, False, False, False,  True,  True, False, False, False,
       False, False, False,  True, False, False,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False, False, False,  True, False,  True, False,  True, False,
        True, False,  True, False, False, False,  True,  True,  True,
        True,  True, False, False, False,  True, False,  True, False,
       False, False,

In [26]:
for i in rng.permutation(len(Xb)):
    print(i)

168
196
156
193
32
122
86
142
135
145
184
98
24
65
107
39
61
174
110
81
68
123
173
74
166
118
136
132
169
199
92
43
106
113
72
5
146
154
62
56
104
186
197
40
180
97
90
0
192
48
37
69
79
161
141
121
47
179
119
41
89
1
28
137
190
153
99
8
73
187
22
103
82
12
60
87
21
194
66
26
198
96
108
149
13
100
176
162
147
49
33
91
155
158
117
63
195
148
34
159
75
130
178
143
52
45
78
115
111
105
183
11
10
15
127
76
88
167
109
175
42
182
27
120
164
29
151
6
44
2
116
7
95
51
64
140
35
129
138
71
57
172
50
25
171
58
19
77
67
126
181
4
31
18
14
53
93
3
85
112
165
80
152
55
101
188
177
36
125
102
20
139
191
9
150
17
83
16
70
128
157
54
59
144
46
38
131
124
133
84
134
189
30
185
23
94
114
170
160
163


In [27]:
x

array([ 0.10061686, -0.37701604,  0.48671773,  1.        ])

In [28]:
W

array([[ 7.07878753e+00,  1.13226924e+01],
       [-3.39260952e+00,  3.25443016e+00],
       [ 1.80384440e+00, -8.69892949e+00],
       [-3.48188980e-03, -1.01328155e+00]])

In [32]:
x @ W > 0

array([ True, False])

In [33]:
t - y

array([0, 0])

In [34]:
t

array([1, 0])

In [35]:
y

array([1, 0])

In [36]:
t - y

array([0, 0])

In [37]:
np.outer(x, (t - y))

array([[ 0.,  0.],
       [-0., -0.],
       [ 0.,  0.],
       [ 0.,  0.]])